In [1]:
%matplotlib inline
######################################################################################################
#
#         SCRIPT DESCRIPTION: Basic Q-Learning in a single intersection
#
#         Input: Map name, route file
#         Output: Tripinfo.xml
#
#####################################################################################################

# Set MAP and time
used_map = 'cross_straight_random'
demand = "random"
# Choice of program (rr = Round Robin, Q = Q-Learning, random)
program = 'Q'
# Q parameters
alpha   = 0.5         # Memory
gamma   = 0.1         # Discount
epsilon = 0.2         # Experimentality


simulation_length = 3600*24*7*4
color_vehicles = False

tracking_length = simulation_length
queue_tracking = True
positional_tracking = False

# Select whether to use previous experience
reloadQm  = False
saveQm    = True

# Set Path for Sumo Home folder
sumo_home_dir = "C:\Program Files (x86)\DLR\Sumo"
    
# Create storage if needed
if positional_tracking == True:
    tracking_data = [[] for _ in range(tracking_length)]
if queue_tracking == True:
    combined_queue = []

# Import basic libraries
import os
import sys
import optparse
import subprocess
import random
import random
import pdb
import xmltodict
import re
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle
from __future__ import absolute_import
from __future__ import print_function

# Current working directory
working_dir = os.getcwd()

# Set environment variable SUMO_HOME
try:
    # Use this line only to manually define SUMO_HOME path
    os.environ["SUMO_HOME"] = sumo_home_dir
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))
except ImportError:
    sys.exit(
        "Please declare environment variable 'SUMO_HOME' as the root directory of your sumo installation (it should contain folders 'bin', 'tools' and 'docs')")

# Import API libraries
from sumolib import checkBinary
import traci

In [2]:
%%javascript
IPython.notebook.kernel.execute('notebook_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [3]:
# Set important directories
networks_dir = "C:\\Users\\acabrejasegea\\OneDrive - The Alan Turing Institute\\Desktop\\ATI\\0_TMF\\MLforFlowOptimisation\\Networks"
scripts_dir = "C:\\Program Files (x86)\\DLR\\Sumo\\tools\\"
runners_dir = "C:\\Users\\CabrejasEgeaAlvaro\\Desktop\\ATI\\05_ML_Traffic_Lights\\MLforFlowOptimisation\\"

notebook = (os.path.splitext(notebook_name))[0]

In [4]:
def keywithmaxval(d):                   #Function used in Example control algorithm
     """ a) create a list of the dict's keys and values; 
         b) return the key with the max value"""  
     v=list(d.values())
     k=list(d.keys())
     return k[v.index(max(v))]

def getEdgeData(edgeIDs):
    #Edge data order (type: double / integer): [CO2, CO, Hydrocarbons, NOx, PMx, Noise,
    #mean Vehicle Length, mean Speed,occupancy, number of vehicles (integer),
    #estimated travel time, Waiting time]
    #http://sumo.dlr.de/daily/pydoc/traci._edge.html 
    data = []

    for i in range(0,len(edgeIDs)):
        data.append(traci.edge.getCO2Emission(edgeIDs[i]))
        data.append(traci.edge.getCOEmission(edgeIDs[i]))
        data.append(traci.edge.getHCEmission(edgeIDs[i]))
        data.append(traci.edge.getNOxEmission(edgeIDs[i]))
        data.append(traci.edge.getPMxEmission(edgeIDs[i]))
        data.append(traci.edge.getNoiseEmission(edgeIDs[i]))
        data.append(traci.edge.getLastStepLength(edgeIDs[i]))
        data.append(traci.edge.getLastStepMeanSpeed(edgeIDs[i]))
        data.append(traci.edge.getLastStepOccupancy(edgeIDs[i]))
        data.append(traci.edge.getLastStepVehicleNumber(edgeIDs[i]))
        data.append(traci.edge.getTraveltime(edgeIDs[i]))
        data.append(traci.edge.getWaitingTime(edgeIDs[i]))
    return data

def getInductionLoopData(inductionLoopIDs):
    #Inductive Loop data order (type: double / integer): [mean Vehicle Length,
    #mean Speed, occupancy, number of vehicles (integer)]
    #http://sumo.dlr.de/daily/pydoc/traci._inductionloop.html 
    data = []

    for i in range(0,len(inductionLoopIDs)):
        data.append(traci.inductionloop.getLastStepLength(inductionLoopIDs[i]))
        data.append(traci.inductionloop.getLastStepMeanSpeed(inductionLoopIDs[i]))
        data.append(traci.inductionloop.getLastStepOccupancy(inductionLoopIDs[i]))
        data.append(traci.inductionloop.getLastStepVehicleNumber(inductionLoopIDs[i]))
    return data

def getLaneData(laneIDs):
    #Lane data order (type: double / integer): [CO2, CO, Hydrocarbons, NOx, PMx, Noise,
    #mean Vehicle Length, mean Speed, occupancy, number of vehicles (integer),
    #estimated travel time, Waiting time]
    #http://sumo.dlr.de/daily/pydoc/traci._lane.html 
    data = []

    for i in range(0,len(laneIDs)):
        data.append(traci.lane.getCO2Emission(laneIDs[i]))
        data.append(traci.lane.getCOEmission(laneIDs[i]))
        data.append(traci.lane.getHCEmission(laneIDs[i]))
        data.append(traci.lane.getNOxEmission(laneIDs[i]))
        data.append(traci.lane.getPMxEmission(laneIDs[i]))
        data.append(traci.lane.getNoiseEmission(laneIDs[i]))
        data.append(traci.lane.getLastStepLength(laneIDs[i]))
        data.append(traci.lane.getLastStepMeanSpeed(laneIDs[i]))
        data.append(traci.lane.getLastStepOccupancy(laneIDs[i]))
        data.append(traci.lane.getLastStepVehicleNumber(laneIDs[i]))
        data.append(traci.lane.getTraveltime(laneIDs[i]))
        data.append(traci.lane.getWaitingTime(laneIDs[i]))
    return data

def getLaneAreaData(laneAreaIDs):
    #Lane area characteristic order (type: double / integer): [jam length (m), jam length (no. vehicles),
    #mean Speed, occupancy, number of vehicles (integer)]
    #http://sumo.dlr.de/daily/pydoc/traci._lanearea.html 
    data = []
    
    for i in range(0,len(laneAreaIDs)):
        data.append(traci.lanearea.getJamLengthMeters(laneAreaIDs[i]))
        data.append(traci.lanearea.getJamLengthVehicle(laneAreaIDs[i]))
        data.append(traci.lanearea.getLastStepMeanSpeed(laneAreaIDs[i]))
        data.append(traci.lanearea.getLastStepOccupancy(laneAreaIDs[i]))
        data.append(traci.lanearea.getLastStepVehicleNumber(laneAreaIDs[i]))
    return data

def getMultiEntryExitData(multiEntryExitIDs):
    #Multi entry exit detection area data order (type: double / integer):
    #mean Speed, number of vehicles (integer)]
    #http://sumo.dlr.de/daily/pydoc/traci._multientryexit.html 
    data = []

    for i in range(0,len(multiEntryExitIDs)):
        data.append(traci.multientryexit.getLastStepMeanSpeed(multiEntryExitIDs[i]))
        data.append(traci.multientryexit.getLastStepVehicleNumber(multiEntryExitIDs[i]))
    return data

def getSimulationData():
    #Simulation data order (type: double):
    #[current time, number of vehicles entering the network]
    #http://sumo.dlr.de/daily/pydoc/traci._simulation.html
    data = []

    data.append(traci.simulation.getCurrentTime())
    data.append(traci.simulation.getDepartedNumber())
    return data

def getTrafficLightData(trafficLightIDs):
    #Traffic light data = [current phase ID, phase duration,
    #current traffic phase]
    #http://sumo.dlr.de/daily/pydoc/traci._trafficlights.html 
    data = []

    for i in range(0,len(trafficLightIDs)):
        data.append(traci.trafficlights.getPhase(trafficLightIDs[i]))
        data.append(traci.trafficlights.getPhaseDuration(trafficLightIDs[i])) #this will reference the fixed time
        data.append(traci.trafficlights.getRedYellowGreenState(trafficLightIDs[i]))
    return data

def getVehicleData(vehicleIDs):
    #Individual vehicle data = [CO2, CO, Hydrocarbons, NOx, PMx, Noise,
    #emission class, total distance travelled (odometer), length,
    #list of upcoming traffic lights [(tlsID, tlsIndex, distance, state), ...], position (m,m),
    #vehicle route (edge IDs),current Speed, Vehicle Class, Waiting Time (below 0.1m/s)]
    #http://sumo.dlr.de/daily/pydoc/traci._vehicle.html 
    data = []

    for i in range(0,len(vehicleIDs)):
        data.append(traci.vehicle.getCO2Emission(vehicleIDs[i]))
        data.append(traci.vehicle.getCOEmission(vehicleIDs[i]))
        data.append(traci.vehicle.getHCEmission(vehicleIDs[i]))
        data.append(traci.vehicle.getNOxEmission(vehicleIDs[i]))
        data.append(traci.vehicle.getPMxEmission(vehicleIDs[i]))
        data.append(traci.vehicle.getNoiseEmission(vehicleIDs[i]))
        data.append(traci.vehicle.getEmissionClass(vehicleIDs[i]))
        data.append(traci.vehicle.getDistance(vehicleIDs[i]))
        data.append(traci.vehicle.getLength(vehicleIDs[i]))
        data.append(traci.vehicle.getNextTLS(vehicleIDs[i]))
        data.append(traci.vehicle.getPosition(vehicleIDs[i]))
        data.append(traci.vehicle.getRoute(vehicleIDs[i]))
        data.append(traci.vehicle.getSpeed(vehicleIDs[i]))
        data.append(traci.vehicle.getVehicleClass(vehicleIDs[i]))
        data.append(traci.vehicle.getWaitingTime(vehicleIDs[i]))
    return data

def getStaticLaneData(laneIDs):
    #Static lane data = [Lane Length, Max Speed]
    #http://sumo.dlr.de/daily/pydoc/traci._lane.html
    data = []

    #Get lane characteristics
    for i in range(0,len(laneIDs)):
        data.append(traci.lane.getLength(laneIDs[i]))
        data.append(traci.lane.getMaxSpeed(laneIDs[i]))
    return data

def getStaticTrafficLightData(trafficLightIDs):
    #Static traffic light data = [lanes controlled by traffic lights, links controlled
    #by traffic lights, ]
    #http://sumo.dlr.de/daily/pydoc/traci._trafficlights.html 
    data = []

    for i in range(0,len(trafficLightIDs)):
        data.append(traci.trafficlight.getControlledLanes(trafficLightIDs[i]))
        data.append(traci.trafficlight.getControlledLinks(trafficLightIDs[i]))
    return data

In [5]:
# Creates a route file for the simulation
def generate_routefile(simulation_length):
    random.seed(0)  # make tests reproducible
    # demand per second from different directions
    if demand == 'fixed':
        pWE = 1. / 8
        pEW = 1. / 8
        pNS = 1. / 15
        pSN = 1. / 15
    elif demand == "random":
        thresholds = np.linspace(0,simulation_length,1+7*4*4)
        pWE_v  = np.linspace(0.04, 0.16, 5)
        pEW_v  = np.linspace(0.04, 0.16, 5)
        pNS_v = np.linspace(0.04, 0.16, 5)
        pSN_v = np.linspace(0.04, 0.16, 5)
    else:
        sys.exit("Error in kind of demand, please choose random or fixed")
    
    # Algorithmic access to routefile (names need to be consistent)
    with open(os.path.join(networks_dir, used_map, (used_map + '.rou.xml')), "w") as routes:
        print("""<routes>
        <vType id="typeWE" accel="0.8" decel="4.5" sigma="0.5" length="5" minGap="2" maxSpeed="25" guiShape="passenger"/>
        <vType id="typeNS" accel="0.8" decel="4.5" sigma="0.5" length="5" minGap="2" maxSpeed="25" guiShape="passenger"/>

        <route id="right" edges="iW oE" />
        <route id="left" edges="iE oW" />
        <route id="down" edges="iN oS" />
        <route id="up" edges="iS oN" />""", file=routes)
        lastVeh = 0
        vehNr = 0
        for i in range(simulation_length):
            if demand == "random":
                if i in thresholds:
                    pWE = random.choice(pWE_v)
                    pEW = random.choice(pEW_v)
                    pNS = random.choice(pNS_v)
                    pSN = random.choice(pSN_v)
                    
            if random.uniform(0, 1) < pWE:
                print('    <vehicle id="right_%i" type="typeWE" route="right" depart="%i" />' % (
                    vehNr, i), file=routes)
                vehNr += 1
                lastVeh = i
            if random.uniform(0, 1) < pEW:
                print('    <vehicle id="left_%i" type="typeWE" route="left" depart="%i" />' % (
                    vehNr, i), file=routes)
                vehNr += 1
                lastVeh = i
            if random.uniform(0, 1) < pNS:
                print('    <vehicle id="down_%i" type="typeNS" route="down" depart="%i" />' % (
                    vehNr, i), file=routes)
                vehNr += 1
                lastVeh = i
            if random.uniform(0, 1) < pSN:
                print('    <vehicle id="down_%i" type="typeNS" route="up" depart="%i" />' % (
                    vehNr, i), file=routes)
                vehNr += 1
                lastVeh = i
        print("</routes>", file=routes)

In [6]:
class NetworkParser:

    def __init__(self, file_path):
        self.junctions = dict()
        self.junction_lights = dict()

        with open(file_path) as f:
            self.file = xmltodict.parse(f.read())

    def get_lane_ids(self):
        for junction in self.file['net']['junction']:
            if junction['@type'] == 'traffic_light':
                self.junctions[junction['@id']] = junction['@incLanes'].split(' ')
        return self.junctions

    # Given a LaneID it returns the traffic light ID
    def get_trafficlights_by_laneID(self, laneID):
        for connection in self.file['net']['connection']:
            if connection['@from'] == laneID.split('_')[0] and '@tl' in connection:
                return connection['@tl']

    def get_junction_trafficlights(self):
        # If the dictionarty for junctions is empty
        if not self.junctions:
            self.get_lane_ids()
        for junctionID in self.junctions.keys():
            for lane in self.junctions[junctionID]:
                traffic_light = self.get_trafficlights_by_laneID(lane)
                if traffic_light in self.junction_lights and lane not in self.junction_lights[traffic_light]:
                    self.junction_lights[traffic_light].append(lane)
                else:
                    self.junction_lights[traffic_light] = []
        return self.junction_lights

    def get_traffic_light(self, laneID):
        if not self.junction_lights:
            self.get_junction_trafficlights()
        for traffic_light, lanes in self.junction_lights.iteritems():
            if laneID in lanes:
                return traffic_light

    def get_phases(self, traffic_light):
        phases = []
        z = 0
        for tls in self.file['net']['tlLogic']:
            if z < 1:
                a=0
            z+=1
            if tls['@id'] == traffic_light:
                for tls_phase in tls['phase']:
                    phases.append(tls_phase)
        return phases

In [7]:
class Qlearner:
    def __init__(self,TLid,lanes,phases):
        self.jid     = TLid
        self.TLid    = TLid
        self.laneids = lanes
        self.Nlanes  = len(self.laneids)                                #len(nodes.junctions[self.jid])
        self.phases  = range(0, 23)                            #traci.trafficlights.getallPhases
        num_phases   = len(self.phases)
        self.Qm      = np.zeros((9, 11))
        
        self.state   = 0
        self.action  = 0
                                
        self.cycle_time = 90
        
        self.historic_reward = list()
        self.historic_queues = [[] for x in range(self.Nlanes)]
  
    def set_action(self,action):                               
        traci.trafficlights.setProgram(self.jid, str(action))
                                                                

    def get_reward(self):
        # Reward defined as a measure of the amount of waiting in the intersection
        return -sum([traci.lane.getLastStepHaltingNumber(lane_id) for lane_id in self.laneids])
 

    def get_state(self,threshold_l=3):
        # State is defined as a combination b/w 5 load level for the 2 routes
        NS_occupancy = traci.lane.getLastStepHaltingNumber("iN_0") + traci.lane.getLastStepHaltingNumber("iS_0")
        EW_occupancy = traci.lane.getLastStepHaltingNumber("iE_0") + traci.lane.getLastStepHaltingNumber("iW_0")

        NS_state = np.trunc(NS_occupancy/8)
        EW_state = np.trunc(EW_occupancy/8)
        
        if NS_state > 2:
            NS_state = 2
        if EW_state >2:
            EW_state = 2
        
        state = int(NS_state + 3* EW_state)

        return(state)

    def update(self,gamma=gamma,alpha=alpha,epsilon=epsilon):
        
        # Get current state, action from class
        state    = self.state
        action   = self.action
        # Generate new action and the reward
        newstate = self.get_state()
        reward   = self.get_reward()
        self.historic_reward.append(reward)

        # Update the Q matrix
        self.Qm[state, action] = self.Qm[state, action]*(1-alpha) + alpha*(reward + gamma*np.max(self.Qm[newstate,:]))

        # Choice of program (Explore or exploit)
        if program == 'Q':
            r=np.random.rand(1)
            if r > epsilon :
                newaction = np.argmax(self.Qm[state,:])
            else:
                newaction = np.random.randint(0, 10)
        elif program == 'rr':
            newaction = 5
        elif program == 'ramdom':
            newaction = np.random.randint(0,10)
        
        # Set variables for next loop and reset cycle timer
        self.state = newstate
        self.action = newaction
                                
        self.set_action(newaction)

In [8]:
def run(learners = []):
    cycle_timer = 0
    timer = 0
    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep()
        timer = timer + 1
        if cycle_timer == 90:
            for learner in learners:
                learner.update()
            cycle_timer = 0
        else:
            cycle_timer = cycle_timer + 1    
        if queue_tracking == True:
            combined_queue.append(traci.lane.getLastStepHaltingNumber(learners[0].laneids[0])+
                                     traci.lane.getLastStepHaltingNumber(learners[0].laneids[1])+
                                     traci.lane.getLastStepHaltingNumber(learners[0].laneids[2])+
                                     traci.lane.getLastStepHaltingNumber(learners[0].laneids[3]))
            for k in range(4):
                learners[0].historic_queues[k].append(traci.lane.getLastStepHaltingNumber(learners[0].laneids[k]))
            
        if positional_tracking == True:
            vehicles = traci.vehicle.getIDList()
            for vehicle in vehicles:
                tracking_data[timer].append(tuple((vehicle, 
                                                    traci.vehicle.getPosition(vehicle),
                                                    traci.vehicle.getCO2Emission(vehicle),
                                                    traci.vehicle.getNOxEmission(vehicle))))
        
        if color_vehicles == True:
            vehicleIDs = traci.vehicle.getIDList()
            waiting_time = list()
            for vehicle in vehicleIDs:
                wait = traci.vehicle.getWaitingTime(vehicle)
                if wait >= 20:
                    traci.vehicle.setColor(vehicle,(255,0,0,255))
                elif wait >= 1:
                    traci.vehicle.setColor(vehicle,(255,255,0,255))
                if persistent_color == False:
                    if wait < 1:
                        traci.vehicle.setColor(vehicle,(0,255,0,255))

    
    sys.stdout.flush()

### ORIGINAL SIMPLE CONTROLLER BELOW ###
# The program looks like this
#    <tlLogic id="0" type="static" programID="0" offset="0">
# the locations of the tls are      NESW
#        <phase duration="31" state="GrGr"/>
#        <phase duration="6"  state="yryr"/>
#        <phase duration="31" state="rGrG"/>
#        <phase duration="6"  state="ryry"/>
#    </tlLogic>
#
#def run():
#    """execute the TraCI control loop"""
#    step = 0
#    # we start with phase 2 where EW has green
#    traci.trafficlight.setPhase("0", 2)
#    while traci.simulation.getMinExpectedNumber() > 0:
#        traci.simulationStep()
#        if traci.trafficlight.getPhase("0") == 2:
#            # we are not already switching
#            if traci.inductionloop.getLastStepVehicleNumber("0") > 0:
#                # there is a vehicle from the north, switch
#                traci.trafficlight.setPhase("0", 3)
#            else:
#                # otherwise try to keep green for EW
#                traci.trafficlight.setPhase("0", 2)
#        step += 1
#    traci.close()
#    sys.stdout.flush()

In [9]:
def get_options():
    optParser = optparse.OptionParser()
    optParser.add_option("--nogui", action="store_true",
                         default=False, help="run the commandline version of sumo")
    options, args = optParser.parse_args()
    return options

In [10]:
# Deployment of Network Parser (input is network file)
npa = NetworkParser(os.path.join(networks_dir, used_map, (used_map + '.net.xml')))

In [11]:
nodes = npa.junctions
phase_list = [len(npa.get_phases(k)) for k in npa.get_junction_trafficlights().keys()]

In [12]:
# Checks
print(nodes)
print(nodes.keys())
print(phase_list)

{'gneJ1': ['iN_0', 'iE_0', 'iS_0', 'iW_0']}
dict_keys(['gneJ1'])
[44]


In [13]:
generate_routefile(simulation_length)

In [14]:
sumoBinary = checkBinary('sumo-gui')
# OR
# sumoBinary = checkBinary('sumo')

In [15]:
learners=[Qlearner(ID,lan,p) for ID, lan, p in zip(nodes.keys(),nodes.values(),phase_list)]

In [16]:
# Define Storage Files Containes
Qm_files = []
Reward_files = []
Queues_files = [[] for x in range(len(learners))]

# Define Storage Files Names
for i in range(len(learners)):
    Qm_files.append(networks_dir+"\\"+used_map+"\\"+"Qm_learner" + str(i) + '_' + demand + '_' + program + ".npy")
    Reward_files.append(networks_dir+"\\"+used_map+"\\"+"Reward_learner" + str(i) + '_' + demand + '_' + program + ".pickle")
    for j in range(learners[i].Nlanes):
        Queues_files[i].append(networks_dir+"\\"+used_map+"\\"+"Queue_learner" + str(i) + '_lane' + str(j) + '_' + demand + '_' + program + ".pickle")

# Load Data from Storage Files
if reloadQm == True:
    for i in range(len(learners)):
        #Load Q-Matrix
        learners[i].Qm = np.load(Qm_files[i])
        print("Loaded pre-trained Q-matrix from file: "+ '\n' + Qm_files[i])
        
        #Load Historic Reward    
        with open(Reward_files[i], 'rb') as fp:
            learners[i].historic_reward = pickle.load(fp)
        print("Loaded Reward history from file: "+'\n' + Reward_files[i])
        
        #Load Historic Queues
        for j in range(learners[i].Nlanes):
            with open(Queues_files[i][j], 'rb') as fp:
                learners[i].historic_queues[j] = pickle.load(fp)
            print("Loaded Reward Queue from file: "+'\n'+ Queues_files[i][j])

In [ ]:
traci.start([sumoBinary, "-c", os.path.join(networks_dir, used_map, (used_map + '.sumocfg')),
                             "--tripinfo-output", os.path.join((notebook + '_out'), 'tripinfo.xml')])
run(learners)

In [ ]:
if saveQm == True:
    for i in range(len(learners)):
        # Save Qm
        np.save(Qm_files[i], learners[i].Qm)
        print("Saved Q-matrix to file: "+'\n'+Qm_files[i])
        
        # Save Historic Reward
        with open(Reward_files[i], 'wb') as fp:
            pickle.dump(learners[i].historic_reward, fp)
        print("Saved Reward history to file: "+'\n' +Reward_files[i])
        
        # Save Historic Queues
        for j in range(learners[i].Nlanes):
            with open(Queues_files[i][j], 'wb') as fp:
                pickle.dump(learners[i].historic_queues[j], fp)
            print("Saved Queue history to file: "+'\n' + Queues_files[i][j])

In [ ]:
learners[0].Qm

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(learners[0].historic_reward)

plt.title("Instantaneous reward over time", fontsize = 20)
plt.xlabel("Time [cycles]"       , fontsize=16)
plt.ylabel("Instantaneous Reward", fontsize=16)

plt.show()

In [ ]:
plt.figure(figsize=(10,7))
styles = ['r-','b-','g-', 'y-']

time = range(len(learners[0].historic_queues[0]))

l1 = plt.plot(time, learners[0].historic_queues[0], styles[0])
l2 = plt.plot(time, learners[0].historic_queues[1], styles[1])
l3 = plt.plot(time, learners[0].historic_queues[2], styles[2])
l4 = plt.plot(time, learners[0].historic_queues[3], styles[3])

plt.title("Queues over time per direction", fontsize = 20)
plt.xlabel("Time [cycles]"       , fontsize=16)
plt.ylabel("Queue Length", fontsize=16)
plt.legend((l1[0],l2[0],l3[0],l4[0]),('North', 'East', 'South', 'West'),
           loc=2, fontsize = "large")

plt.show()

In [ ]:
thresholds = np.linspace(0,simulation_length,1+7*4*4)
thresholds

In [ ]:
len(learners[0].historic_queues)

In [ ]:
len(learners[0].historic_queues)

In [ ]:
[*nodes][0]

In [ ]:
nodes[str([*nodes][0])]

In [ ]:
len(nodes[str([*nodes][0])])

In [ ]:
learners[0].historic_reward

In [ ]:
with open(networks_dir+'\\'+used_map+"\\"+"Reward_" + str(0) + demand + '_' + program + ".pickle", 'rb') as fp:
    a = pickle.load(fp)
print("Loaded Reward history from file: "+'\n'+networks_dir+"\\"+used_map+"\\"+"Reward_" + str(0) + demand + '_' + program + ".pickle")

In [ ]:
a

In [ ]:
learners[0].historic_queues

In [ ]:
learners[0].historic_queues[0] = [0,0,0,0]
learners[0].historic_queues[1] = [1,1,1,1]
learners[0].historic_queues[2] = [2,2,2,2]
learners[0].historic_queues[3] = [3,3,3,3]
learners[0].historic_reward    = [4,4,4,4]
learners[0].Qm                 = [5,5,5,5]

In [ ]:
for i in range(4):
    print(learners[0].historic_queues[i])
print(learners[0].historic_reward)
print(learners[0].Qm)

In [ ]:
temporary

In [ ]:
Queues_files = [[] for x in range(2)]
Queues_files[1]

In [ ]:
reloadQm = True